In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import matplotlib.pyplot as plt
from botorch.utils.sampling import draw_sobol_samples 
import numpy as np 
import json

import sys
sys.path.append('..')
import sample_utilities.samples as samples
import uuid

import scipy
from scipy.stats.qmc import Sobol
from scipy.stats import qmc

import pandas as pd

## Load bounds

In [3]:
target_volume = 10000

In [4]:
with open('Mesoporous_constants_APS.json', 'r') as f:
    constants = json.load(f)


ctab_lower_vf = constants['ctab']['minimum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume  
f127_lower_vf = constants['f127']['minimum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

ctab_upper_vf = constants['ctab']['maximum_mass']/constants['ctab']['stock_concentration_mg_uL']/target_volume
f127_upper_vf = constants['f127']['maximum_mass']/constants['f127']['stock_concentration_mg_uL']/target_volume

lower_bounds = [constants['TEOS']['minimum_volume_fraction'],
                constants['ammonia']['minimum_volume_fraction'],
                constants['ethanol']['minimum_volume_fraction'],
                constants['ctab']['minimum_mass'],
                constants['f127']['minimum_mass']]

upper_bounds = [constants['TEOS']['maximum_volume_fraction'],
                constants['ammonia']['maximum_volume_fraction'],
                constants['ethanol']['maximum_volume_fraction'],
                constants['ctab']['maximum_mass'],
                constants['f127']['maximum_mass']]

In [5]:
ctab_upper_vf

0.6666666666666667

In [6]:
def constrained_sobol_sampling_rejection(n: int, num_samples: int, m_samples = 8, seed: int = None):
    """
    Generates Sobol samples in n dimensions satisfying:
    a) Sum of all components = 1.0
    b) Last component (nth dim) is in [0.4, 1.0]

    Uses rejection sampling: Generates a large pool and filters valid samples.

    Args:
        n (int): Number of dimensions.
        num_samples (int): Number of valid samples to return.
        pool_size (int): Number of initial samples to generate before filtering.
        seed (int, optional): Random seed for reproducibility.

    Returns:
        torch.Tensor: (num_samples, n) tensor of valid samples.
    """
    if seed is not None:
        torch.manual_seed(seed)

    valid_samples = []
    
    while len(valid_samples) < num_samples:
        print('Number valid samples: ', len(valid_samples))
        # Generate Sobol samples in [0,1]^n
        #sobol_samples = draw_sobol_samples(bounds=torch.tensor([[0.0] * n, [1.0] * n]), n=pool_size, q=1).squeeze(1)
        sampler = Sobol(d=n, seed = seed)
        sampled_points = sampler.random_base2(m_samples)
    
        sampled_volume_fractions = qmc.scale(sampled_points, lower_bounds, upper_bounds)


        for i in range(len(sampled_volume_fractions)):
            teos_vol_frac = sampled_volume_fractions[i, 0]
            ammonia_vol_frac = sampled_volume_fractions[i, 1]
            ethanol_vol_frac = sampled_volume_fractions[i, 2]
            # convert surfactant mass to volumes
            ctab_mass = sampled_volume_fractions[i, 3]
            f127_mass = sampled_volume_fractions[i, 4]
    
            ctab_vol_frac = ctab_mass/constants['ctab']['stock_concentration_mg_uL']/target_volume
            f127_vol_frac = f127_mass/constants['f127']['stock_concentration_mg_uL']/target_volume
    
            water_from_surfactant_vf = ctab_vol_frac+f127_vol_frac
    
            
    
            water_min_vf = constants['water']['minimum_volume_fraction']
    
            # water that needs to be delivered to meet minimum volume fraction
            water_min_vf_delivered = max(water_min_vf-water_from_surfactant_vf, 0)
    
            max_other_components = 1 - water_min_vf_delivered

            print(max_other_components)
    
    
            sum_nonwater_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac + ctab_vol_frac + f127_vol_frac
            sum_water_free_additions = teos_vol_frac + ammonia_vol_frac + ethanol_vol_frac
            if sum_nonwater_additions < max_other_components:

                water_vol_frac = 1 - sum_water_free_additions
                sample = [teos_vol_frac, ammonia_vol_frac, ethanol_vol_frac, ctab_mass, f127_mass, water_vol_frac]
                
                print('found valid sample: ', sample)
                valid_samples.append(sample)

    return valid_samples[0:num_samples]


In [7]:
baseline_samples = constrained_sobol_sampling_rejection(5, 32, m_samples= 7)

Number valid samples:  0
1
found valid sample:  [np.float64(0.01897037719422951), np.float64(0.018937377342954278), np.float64(0.23302871568128466), np.float64(53.0969182215631), np.float64(110.36637229844928), np.float64(0.7290635297815315)]
1
0.9889883297185103
found valid sample:  [np.float64(0.045622458941768855), np.float64(0.03412858731113374), np.float64(0.28273111711256205), np.float64(12.08582654595375), np.float64(154.20807637274265), np.float64(0.6375178366345353)]
1
1
1
found valid sample:  [np.float64(0.03289529821276665), np.float64(0.03807605862617493), np.float64(0.208597372751683), np.float64(31.746393982321024), np.float64(205.91566935181618), np.float64(0.7204312704093754)]
1
1
found valid sample:  [np.float64(0.031298181463964285), np.float64(0.052569298576563596), np.float64(0.25876802171580493), np.float64(33.42808824032545), np.float64(133.6501962505281), np.float64(0.6573644982436672)]
1
1
0.9023365354786317
found valid sample:  [np.float64(0.036816389657557014)

In [8]:
for sample in baseline_samples:
    sample_sum = 0
    for entry in sample:
        sample_sum += entry
    print('Sample sum: ', sample_sum)


Sample sum:  164.46329052001238
Sample sum:  167.2939029186964
Sample sum:  238.66206333413717
Sample sum:  168.07828449085355
Sample sum:  53.11666730232537
Sample sum:  88.64680097810924
Sample sum:  104.91664855182171
Sample sum:  131.8377000875771
Sample sum:  159.7927943840623
Sample sum:  123.54213666543365
Sample sum:  18.785485638305545
Sample sum:  78.57720691151917
Sample sum:  84.33855370990932
Sample sum:  56.12342908419669
Sample sum:  156.13686353340745
Sample sum:  113.68144119530916
Sample sum:  81.62430730089547
Sample sum:  95.99724723398685
Sample sum:  95.65655111707747
Sample sum:  240.05283282510936
Sample sum:  222.46294393576682
Sample sum:  91.69363798014821
Sample sum:  238.52677220851183
Sample sum:  163.1745251864195
Sample sum:  127.38437995687127
Sample sum:  199.31842528656125
Sample sum:  161.1376400142908
Sample sum:  134.31920778378844
Sample sum:  246.1862447336316
Sample sum:  201.40822820737958
Sample sum:  94.87764010578394
Sample sum:  130.9379470

In [9]:
len(baseline_samples)

32

In [10]:
uuid_vals = [uuid.uuid4() for val in baseline_samples]
teos_volume = [target_volume*entry[0] for entry in baseline_samples]

In [11]:
samples_gen = []
for i in range(len(baseline_samples)):
    row = baseline_samples[i]
    sample = samples.MesoporousSample(target_volume=target_volume, reactant_fp='Mesoporous_constants_APS.json', teos_vol_frac=row[0], ammonia_vol_frac=row[1], ethanol_vol_frac=row[2], ctab_mass=row[3], f127_mass=row[4], water_vol_frac=row[5])
    sample.calculate_reactant_volumes()
    #sample.calculate_silica_mass_concentration()
    #sample.calculate_silica_mass_fraction()
    #sample.calculate_dilution_volumefraction(target_dilution)
    samples_gen.append(sample)
    print('final etoh vol: ', sample.ethanol_volume)

TEOS ETOH:  350.95197809324594
Solvent etoh total:  350.95197809324594
final etoh vol:  1979.3351787196007
TEOS ETOH:  844.0154904227238
Solvent etoh total:  844.0154904227238
final etoh vol:  1983.2956807028968
TEOS ETOH:  608.563016936183
Solvent etoh total:  608.563016936183
final etoh vol:  1477.410710580647
TEOS ETOH:  579.0163570833392
Solvent etoh total:  579.0163570833392
final etoh vol:  2008.66386007471
TEOS ETOH:  681.1032086648049
Solvent etoh total:  681.1032086648049
final etoh vol:  2300.574250049889
TEOS ETOH:  771.6168482813518
Solvent etoh total:  771.6168482813518
final etoh vol:  3002.683509231778
TEOS ETOH:  449.9424967023078
Solvent etoh total:  449.9424967023078
final etoh vol:  2018.912866241997
TEOS ETOH:  321.5435181348584
Solvent etoh total:  321.5435181348584
final etoh vol:  2721.331678556744
TEOS ETOH:  85.41703505930491
Solvent etoh total:  85.41703505930491
final etoh vol:  3758.2921962949913
TEOS ETOH:  480.1364863910712
Solvent etoh total:  480.1364863

In [12]:
sample.water_volume

np.float64(3463.890027677019)

In [13]:
uuid_vals = []
teos_volumes = []
ammonia_volumes = []
ethanol_volumes = []
water_volumes = []
ctab_volumes = []
f127_volumes = []
sample_names = []

for i, sample in enumerate(samples_gen):
    sample_names.append(f'SobolBaseline_{i+1}')
    uuid_vals.append(uuid.uuid4())
    teos_volumes.append(sample.teos_volume)
    ammonia_volumes.append(sample.ammonia_volume)
    ethanol_volumes.append(sample.ethanol_volume)
    water_volumes.append(sample.water_volume)
    ctab_volumes.append(sample.ctab_volume)
    f127_volumes.append(sample.f127_volume)\

    total_volume = sample.teos_volume+sample.ammonia_volume + sample.water_volume + sample.ethanol_volume + sample.ctab_volume + sample.f127_volume
    print(f'Total volume sample SobolBaseline_{i+1}: {total_volume}')



Total volume sample SobolBaseline_1: 10000.0
Total volume sample SobolBaseline_2: 10000.0
Total volume sample SobolBaseline_3: 10000.0
Total volume sample SobolBaseline_4: 10000.0
Total volume sample SobolBaseline_5: 10000.0
Total volume sample SobolBaseline_6: 10000.0
Total volume sample SobolBaseline_7: 10000.0
Total volume sample SobolBaseline_8: 10000.0
Total volume sample SobolBaseline_9: 10000.0
Total volume sample SobolBaseline_10: 10000.0
Total volume sample SobolBaseline_11: 9999.999999999998
Total volume sample SobolBaseline_12: 9999.999999999998
Total volume sample SobolBaseline_13: 10000.0
Total volume sample SobolBaseline_14: 10000.0
Total volume sample SobolBaseline_15: 10000.0
Total volume sample SobolBaseline_16: 10000.0
Total volume sample SobolBaseline_17: 10000.0
Total volume sample SobolBaseline_18: 10000.0
Total volume sample SobolBaseline_19: 10000.0
Total volume sample SobolBaseline_20: 10000.0
Total volume sample SobolBaseline_21: 10000.0
Total volume sample Sob

In [14]:
sample_table = pd.DataFrame({'sample_name':sample_names,
                             'uuid':uuid_vals,
                             'teos_volume':teos_volumes,
                             'ammonia_volume':ammonia_volumes,
                             'water_volume':water_volumes,
                             'ethanol_volume':ethanol_volumes,
                             'ctab_volume':ctab_volumes,
                             'F127_volume':f127_volumes
                            })

In [15]:
sample_table

,sample_name,uuid,teos_volume,ammonia_volume,water_volume,ethanol_volume,ctab_volume,F127_volume
0,SobolBaseline_1,cd773fee-cd28-4355-8f4c-a1b6ead07914,540.655750,189.373773,1543.513304,1979.335179,3539.794548,2207.327446
1,SobolBaseline_2,050350bb-9ee1-41f9-99b2-29b8909480aa,1300.240080,341.285873,2485.295069,1983.295681,805.721770,3084.161527
2,SobolBaseline_3,4e917120-dc49-4ce6-b8a0-9fbe5194ace7,937.515999,380.760586,969.573052,1477.410711,2116.426265,4118.313387
3,SobolBaseline_4,4b55032c-b883-4661-a831-ac48b742ed57,891.998172,525.692986,1672.101841,2008.663860,2228.539216,2673.003925
4,SobolBaseline_5,5f618249-5d38-45fb-bd62-bc3f41b53083,1049.267105,210.547688,3416.245602,2300.574250,2830.045727,193.319628
5,SobolBaseline_6,cb2077d4-81c8-4203-afec-f32b61f259a2,1188.707037,499.778263,1440.898646,3002.683509,3021.423609,846.508937
6,SobolBaseline_7,9e079ae2-dd06-48bb-80ae-4df326baa3a7,693.154657,262.624437,1472.503550,2018.912866,4963.530740,589.273749
7,SobolBaseline_8,152ac6d3-06eb-4a29-af6b-e5061df4e324,495.350825,482.903414,2348.633956,2721.331679,1907.180178,2044.599948
8,SobolBaseline_9,bea8b088-a3d4-4b35-bf7f-4db6103f35d3,131.588405,194.222158,2062.636674,3758.292196,967.720969,2885.539597
9,SobolBaseline_10,021da2dc-f44c-44df-b7c0-806a13e368b9,739.669722,427.068651,1856.721283,1706.732498,4027.093018,1242.714828


In [16]:
vial_volume = 17000
cols = ['teos_volume', 'ammonia_volume', 'ethanol_volume', 'water_volume', 'ctab_volume', 'F127_volume']
for col in cols:
    total_volume = sample_table.iloc[0:24][col].sum()
    n_vials = total_volume/vial_volume
    print(f'Volume for {col}: {total_volume}, vials required: {n_vials}')

Volume for teos_volume: 22572.286400797544, vials required: 1.3277815529880908
Volume for ammonia_volume: 8031.154810078442, vials required: 0.47242087118108483
Volume for ethanol_volume: 52521.87989891716, vials required: 3.0895223469951274
Volume for water_volume: 53662.21910806838, vials required: 3.1566011240040224
Volume for ctab_volume: 58498.31148609519, vials required: 3.4410771462408936
Volume for F127_volume: 44714.14829604328, vials required: 2.6302440174143102


In [17]:
sample_table.to_csv('Mesoporous_SobolBaseline_APS_RestrictedAmmoniaTEOS_3_28_25.csv')